In [1]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [2]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
# pattern = r'[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?<=[a-z])(?=[A-Z])'
pattern = r'[\n\r]+|(?<=[a-z]\.)|(?<=[a-z])'


In [3]:
import os
import sys
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import googletrans
from pathlib import Path
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [4]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_dropped.pkl').reset_index(drop=True)


In [5]:
df_jobs = df_jobs[:100]

In [6]:
# df_jobs = df_jobs.sample(n=100, random_state=1)

In [7]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_glob_paths_10.pkl').reset_index(drop=True)


In [8]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     100 non-null    object 
 1   Platform           100 non-null    object 
 2   Job ID             100 non-null    object 
 3   Job Title          100 non-null    object 
 4   Company Name       100 non-null    object 
 5   Location           100 non-null    object 
 6   Job Description    100 non-null    object 
 7   Rating             100 non-null    float64
 8   Employment Type    100 non-null    object 
 9   Company URL        100 non-null    object 
 10  Job URL            100 non-null    object 
 11  Job Age            100 non-null    object 
 12  Job Age Number     100 non-null    object 
 13  Collection Date    100 non-null    object 
 14  Data Row           0 non-null      float64
 15  Tracking ID        0 non-null      object 
 16  Industry           0 non-nu

In [9]:
df_jobs.columns


Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name',
       'Location', 'Job Description', 'Rating', 'Employment Type',
       'Company URL', 'Job URL', 'Job Age', 'Job Age Number',
       'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date',
       'Type of ownership'],
      dtype='object')

In [10]:
str_fix_incl = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'
str_fix_eg = 'Partner with Procurement in order to manage suppliers for the projects & programs in scope, e.g.'



In [11]:
len(df_jobs.loc[(df_jobs['Job Description'].str.contains(str_fix_incl)) | (df_jobs['Job Description'].str.contains(str_fix_eg))])


1

In [12]:
# job_descriptions = list(set(df_jobs_incl['Job Description'].to_list()))


In [13]:
job_descriptions = list(
    set(
        df_jobs['Job Description'].loc[
            (df_jobs['Job Description'].str.contains(str_fix_incl)) |
            (df_jobs['Job Description'].str.contains(str_fix_eg))
        ].to_list()
    )
)


In [14]:
len(job_descriptions)


1

In [15]:
# Load NLK
from nltk.tokenize import sent_tokenize, word_tokenize
nltk_path = f'{llm_path}/nltk'
nltk.data.path.append(nltk_path)

nltk.download('words', download_dir = nltk_path)
nltk.download('punkt', download_dir = nltk_path)
nltk.download('stopwords', download_dir = nltk_path)


[nltk_data] Downloading package words to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
job_descriptions[0][1500:2000]

'suppliers for the projects & programs in scope, e.g. interviews, staffing, supplier costs.\n\nCommunication / stakeholder management\nAct as the key contact person for strategic stakeholders.\nApply appropriate and effective communication methods to senior management and important stakeholders (incl. vendor) throughout the project lifecycle.\nAs conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.\nDrive change management act'

In [17]:
# pattern = r'[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?<=[a-z])(?=[A-Z])'
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'



In [18]:
# nltk_sentencizer = []

# for job_description in job_descriptions:
#     for sentence in sent_tokenize(job_description):
#         for sent in re.split(pattern, sentence):
#             nltk_sentencizer.append(sent)

In [123]:
nltk_sentencizer = [
    sent 
    for job_description in job_descriptions 
    for sentence in sent_tokenize(job_description) 
    for sent in re.split(pattern, sentence)
    if len(sent) != 0
]

In [120]:
test_nltk_sentencizer = [
    sentence 
    for job_description in job_descriptions 
    for sentence in sent_tokenize(job_description) 
#     for sent in re.split(pattern, sentence)
    if len(sent) != 0
]

In [121]:
len(test_nltk_sentencizer)

26

In [122]:
next(idx for idx, sent in enumerate(test_nltk_sentencizer) if 'Power' in sent)

25

In [118]:
t[25]

'3+ years of experience in leading a team\n\nSoft-Skills\nVery good communication skills, especially when interacting with different levels of business\nAbility to work in a fast-paced environment with different international cultures\nAbility to handle ambiguity and untangle complex situations into actionable activities\nDistinctive strategic mindset and ability to prioritize and delegate high numbers of tasks with varying workload and importance\nSolutions-oriented approach and entrepreneurial mindset\nGood numerical and analytical skills\nHighly developed leadership skills are required\n\nHard-Skills\nKnowledge of digital technologies and communications platforms and culture\nStrong MS-Office skills (Word, Excel, PowerPoint)\nAbility to travel, domestic or international, as required\nIn-depth understanding of IT\nFluent English both verbally and written'

In [80]:
# nltk_sentencizer

In [114]:
len(nltk_sentencizer)

70

In [82]:
for idx, sent in enumerate(nltk_sentencizer):
    if str_fix_eg in sent:
        print(idx)

18


In [74]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'

for idx, sent in enumerate(nltk_sentencizer):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

22


In [107]:
next(idx for idx, sent in enumerate(nltk_sentencizer) if str_fix_eg in sent)

18

In [108]:
# sentences split on e.g. and incl.
nltk_sentencizer[18:24]

['Partner with Procurement in order to manage suppliers for the projects & programs in scope, e.g.',
 'interviews, staffing, supplier costs.',
 'Communication / stakeholder management',
 'Act as the key contact person for strategic stakeholders.',
 'Apply appropriate and effective communication methods to senior management and important stakeholders (incl.',
 'vendor) throughout the project lifecycle.']

In [76]:
for idx, sent in enumerate(nltk_sentencizer):
    if 'Power' in sent:
        print(idx)

66


In [111]:
next(idx for idx, sent in enumerate(nltk_sentencizer) if 'Power' in sent)

66

In [77]:
nltk_sentencizer[66]

'Strong MS-Office skills (Word, Excel, PowerPoint)'

In [78]:
for idx, sentence in enumerate(nltk_sentencizer[18:24]):
    print(f'Sentence {idx+1}: {sentence}\n')

Sentence 1: Partner with Procurement in order to manage suppliers for the projects & programs in scope, e.g.

Sentence 2: interviews, staffing, supplier costs.

Sentence 3: Communication / stakeholder management

Sentence 4: Act as the key contact person for strategic stakeholders.

Sentence 5: Apply appropriate and effective communication methods to senior management and important stakeholders (incl.

Sentence 6: vendor) throughout the project lifecycle.



In [26]:
# Function to make a list of punctuations that determine sentence boundry, i.e., split characters
def make_custom_punct_chars(main_punct_chars = [':', '|'], repeated_punct_chars = ['\n', ',']):
    custom_punct_chars = []
    temp_multi = []
    temp_spaced = []

    for punct_char in main_punct_chars:
        custom_punct_chars+= f'{punct_char}', f'{punct_char} '

    for idx in range(4):
        for punct_char in repeated_punct_chars:
            temp_multi.append(f'{punct_char}'*int(idx+1))
            temp_spaced.append(f'{punct_char} '*int(idx+1))

    for multi, spaced in zip(temp_multi, temp_spaced):
        custom_punct_chars+= multi, spaced

    custom_punct_chars.remove(',')
    custom_punct_chars.remove(', ')

    return custom_punct_chars


In [27]:
custom_punct_chars = make_custom_punct_chars()

In [28]:
custom_punct_chars

[':',
 ': ',
 '|',
 '| ',
 '\n',
 '\n ',
 '\n\n',
 '\n \n ',
 ',,',
 ', , ',
 '\n\n\n',
 '\n \n \n ',
 ',,,',
 ', , , ',
 '\n\n\n\n',
 '\n \n \n \n ',
 ',,,,',
 ', , , , ']

In [29]:
# Set up Spacy
import spacy
from spacy.symbols import ORTH, LEMMA

nlp = spacy.load('en_core_web_sm')


In [30]:
sentencizer = nlp.add_pipe('sentencizer')
sentencizer.punct_chars.update(custom_punct_chars)


In [31]:
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    with open(f'{data_dir}punctuations.txt', 'wb') as f:
        pickle.dump(sentencizer.punct_chars, f)

In [32]:
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_char = pickle.load(f)

In [33]:
# special_cases = {":)": [{"ORTH": ":)"}]}
# prefix_re = re.compile(r'''^[\\[\\("']''')
# suffix_re = re.compile(r'''[\\]\\)"']$''')
# infix_re = re.compile(r'''[-~]''')
# simple_url_re = re.compile(r'''^https?://''')

# def custom_tokenizer(nlp):
#     return Tokenizer(nlp.vocab, rules=special_cases,
#                                 prefix_search=prefix_re.search,
#                                 suffix_search=suffix_re.search,
#                                 infix_finditer=infix_re.finditer,
#                                 url_match=simple_url_re.match)

In [34]:
# nlp = spacy.load('en_core_web_sm')
# nlp.tokenizer.add_special_case('incl.', [{ORTH: 'incl', LEMMA: 'including'}])


In [35]:
# from spacy.symbols import ORTH, LEMMA

special_cases_dict = {
    'incl.': [{65: 'incl', 67: 'including'}],
    'incl. ': [{65: 'incl', 67: 'including'}],
    '(incl.': [{65: 'incl', 67: 'including'}],
    'etc.': [{65: 'etc', 67: 'et cetera'}],
    'etc. ': [{65: 'etc', 67: 'et cetera'}],
    'e.g.': [{65: 'e.g', 67: 'for example'}],
    'e.g. ': [{65: 'e.g', 67: 'for example'}],
}

nlp.tokenizer.rules.update(special_cases_dict)


In [36]:
nlp.tokenizer.rules['e.g.']

[{65: 'e.g', 67: 'for example'}]

In [37]:
# [y for x in a for y in [x[0]] * x[1]]


In [84]:
# # Spacy sentencizer
# spacy_sentencizer = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_description in job_descriptions:
#         for sentence in nlp(job_description).sents:
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencizer.append(sent)
# #         doc = nlp(job_description)
# #         spacy_sentencizer.extend([sent.text for sent in nlp(job_description).sents])


In [169]:
# Spacy tokenizer
spacy_tokenizer = []

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    for job_sentence in spacy_sentencizer:
        for token in nlp.tokenizer(job_sentence):
            if len(token.text) > 1 and token.text not in custom_punct_chars and not token.is_stop:
                spacy_tokenizer.append(token.text)

In [170]:
type(spacy_tokenizer[0])

str

In [171]:
len(spacy_tokenizer)

488

In [172]:
spacy_tokenizer

['Purpose',
 'Studio',
 'Director',
 'Special',
 'Projects',
 'adidas',
 'Digital',
 'task',
 'manage',
 'deliver',
 'Studio',
 'programs',
 'including',
 'projects',
 'roadmap',
 'program',
 'scope',
 'comprise',
 'multiple',
 'countries',
 'functions',
 'objective',
 'conduct',
 'projects',
 'boundaries',
 'time',
 'cost',
 'quality',
 'focusing',
 'consumers',
 'expectations',
 'requirements',
 'Planning',
 'executing',
 'tracking',
 'projects',
 'programs',
 'cooperation',
 'respective',
 'functions',
 'project',
 'team',
 'members',
 'examples',
 'expected',
 'role',
 'Additionally',
 'focusing',
 'resource',
 'planning',
 'vendor',
 'management',
 'coordination',
 'activities',
 'acting',
 'central',
 'interface',
 'key',
 'stakeholders',
 'Scope',
 'Manage',
 'deliver',
 'Studio',
 'programs',
 'including',
 'projects',
 'roadmap',
 'Project/',
 'Program',
 'Delivery',
 'Manage',
 'set',
 'roll',
 'Studio',
 'hubs',
 'Portland',
 'Shanghai',
 'locations',
 'Ensure',
 'clear',
 '

In [173]:
# Spacy tokenizer
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    spacy_tokenizer = [
        token.text 
        for job_sentence in spacy_sentencizer 
        for token in nlp.tokenizer(job_sentence) 
        if len(token.text) > 1 and token.text not in custom_punct_chars and not token.is_stop
    ]

In [174]:
type(spacy_tokenizer[0])

str

In [175]:
len(spacy_tokenizer)

488

In [176]:
spacy_tokenizer

['Purpose',
 'Studio',
 'Director',
 'Special',
 'Projects',
 'adidas',
 'Digital',
 'task',
 'manage',
 'deliver',
 'Studio',
 'programs',
 'including',
 'projects',
 'roadmap',
 'program',
 'scope',
 'comprise',
 'multiple',
 'countries',
 'functions',
 'objective',
 'conduct',
 'projects',
 'boundaries',
 'time',
 'cost',
 'quality',
 'focusing',
 'consumers',
 'expectations',
 'requirements',
 'Planning',
 'executing',
 'tracking',
 'projects',
 'programs',
 'cooperation',
 'respective',
 'functions',
 'project',
 'team',
 'members',
 'examples',
 'expected',
 'role',
 'Additionally',
 'focusing',
 'resource',
 'planning',
 'vendor',
 'management',
 'coordination',
 'activities',
 'acting',
 'central',
 'interface',
 'key',
 'stakeholders',
 'Scope',
 'Manage',
 'deliver',
 'Studio',
 'programs',
 'including',
 'projects',
 'roadmap',
 'Project/',
 'Program',
 'Delivery',
 'Manage',
 'set',
 'roll',
 'Studio',
 'hubs',
 'Portland',
 'Shanghai',
 'locations',
 'Ensure',
 'clear',
 '

In [95]:
for idx, sent in enumerate(spacy_sentencizer):
    if str_fix_eg in sent:
        print(idx)

19


In [105]:
next(idx for idx, sent in enumerate(spacy_sentencizer) if str_fix_eg in sent)

19

In [96]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'


for idx, sent in enumerate(spacy_sentencizer):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

22


In [106]:
next(idx for idx, sent in enumerate(spacy_sentencizer) if str_fix_incl.split('\(')[0] in sent)

22

In [127]:
spacy_sentencizer[19:23]

['Partner with Procurement in order to manage suppliers for the projects & programs in scope, e.g. interviews, staffing, supplier costs.',
 'Communication / stakeholder management',
 'Act as the key contact person for strategic stakeholders.',
 'Apply appropriate and effective communication methods to senior management and important stakeholders (incl.']

In [109]:
for idx, sent in enumerate(spacy_sentencizer):
    if 'Power' in sent:
        print(idx)

66


In [110]:
next(idx for idx, sent in enumerate(spacy_sentencizer) if 'Power' in sent)

66

In [128]:
spacy_sentencizer[66]

'Strong MS-Office skills (Word, Excel, PowerPoint)'

In [129]:
for idx, sentence in enumerate(spacy_sentencizer[22:28]):
    print(f'Sentence {idx+1}: {sentence}\n')

Sentence 1: Apply appropriate and effective communication methods to senior management and important stakeholders (incl.

Sentence 2: vendor) throughout the project lifecycle.

Sentence 3: As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.

Sentence 4: Drive change management activities for respective projects & programs and ensure changes are smoothly and successfully implemented to achieve lasting benefits.

Sentence 5: Project Controlling

Sentence 6: Manage project controlling as an independent element to ensure that actual project costs are in line the committed budget.



In [ ]:
# Spacy tokenizer
spacy_tokenizer = []

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    for job_sentence in spacy_sentencizer:
#         doc = nlp.tokenizer(job_sentence)
        spacy_tokenizer.extend(
            [
                token.text for token in nlp.tokenizer(job_sentence) 
                if token.text not in custom_punct_chars
                and not token.is_stop 
                
            ]
        )

In [ ]:
len(spacy_tokenizer)

In [ ]:
spacy_tokenizer[:10]

In [ ]:
# from spacy.lang.en import English
# nlp = English()
# nlp.add_pipe('sentencizer')
doc = nlp('Operating income incl. JV was SEK 2.1 b. with an operating margin of 4.0%')

In [ ]:
[s for s in doc.sents]

In [ ]:
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_trf')

matcher = Matcher(nlp.vocab)

bigram_rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

trigram_patterns = [[{"POS": i} for i in j] for j in rules]


In [ ]:
patterns

In [ ]:
patterns = {
    'noun_verb': [{'POS': 'NOUN'}, {'POS': 'VERB'}],
    'verb_noun': [{'POS': 'VERB'}, {'POS': 'NOUN'}],
    'adj_noun': [{'POS': 'ADJ'}, {'POS': 'NOUN'}],
    'adj_propn': [{'POS': 'ADJ'}, {'POS': 'PROPN'}],
}

In [ ]:
for pattern_name, pattern in patterns.items():
    matcher.add(pattern_name, [pattern])


In [ ]:
for job_description in job_descriptions:
    doc = nlp(job_description)
    matches = matcher(doc)

In [ ]:
for match_id, start, end in matches:
    
    # Get string representation
    string_id = nlp.vocab.strings[match_id]

    # The matched span
    span = doc[start:end]
    
    print(repr(span.text))
    print(match_id, string_id, start, end)
    print()


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [ ]:
# for sentence in spacy_sentencizer:
#     print(model.encode(sentence))



In [ ]:
from transformers.models.electra.modeling_tf_electra import TFElectraMainLayer

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("pavanchhatpar/electra-base-sentence-splitter")

model = AutoModelForPreTraining.from_pretrained("pavanchhatpar/electra-base-sentence-splitter", from_tf=True)


In [ ]:
nlp = pipeline('sentence-splitter', model)